In [2]:
# uncomment to install pyannote audio
# !pip install pyannote.audio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 MB 5.6 MB/s eta 0:00:000m eta 0:00:010:00:01
  Using cached numpy-1.25.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.9/612.9 KB 3.7 MB/s eta 0:00:000:00:010:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.0
    Uninstalling numpy-1.26.0:
      Successfully uninstalled numpy-1.26.0
  Attempting uninstall: anyio
    Found existing installation: anyio 4.0.0
    Uninstalling anyio-4.0.0:
      Successfully uninstalled anyio-4.0.0


In [9]:
from pyannote.audio import Pipeline
from pyannote.core import Annotation, Segment
from os.path import basename, splitext
import glob
import pandas as pd

In [4]:
# replace with corpus wav path on your local machine
wavdir = '/tmp/wav/'

In [8]:
def refdur(base):
    df = pd.read_csv('../../annotations/vad/%s.csv' % base)
    return df.stop[len(df) -1]

In [7]:
# you may need a hugginface token (i won't give mine) and set option use_auth_token
pipeline = Pipeline.from_pretrained("pyannote/voice-activity-detection")

Lightning automatically upgraded your loaded checkpoint from v1.1.3 to v2.0.9.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../../../../.cache/torch/pyannote/models--pyannote--segmentation/snapshots/059e96f964841d40f1a5e755bb7223f76666bba4/pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.0.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.7.1, yours is 2.1.0+cu121. Bad things might happen unless you revert torch to 1.x.


In [16]:
for f in glob.glob('../../annotations/vad/*.csv'):
    print(f)
    base = basename(splitext(f)[0])
    dur = refdur(base)
    annotation = Annotation()
    annotation[Segment(0, dur)] = 'nonspeech'

    output = pipeline('%s/%s.wav' % (wavdir, base))

    for seg in output.itersegments():
        if seg.start < .05:
            seg = Segment(0, seg.end)
        if abs(seg.end - dur)  < .05:
            seg = Segment(seg.start, dur)
        annotation = annotation.extrude(seg)
        annotation[seg] = 'speech'
    anotation = annotation.support(collar = 0.1)

    
    lrec = []
    for seg, _, label in annotation.itertracks(yield_label=True):
        lrec.append((seg.start, seg.end, label))
    df = pd.DataFrame.from_records(lrec, columns=['start', 'stop', 'label'])
    df.to_csv('../../automatic_baselines/pyannote_vad/%s.csv' % base, index=False)
    print(lrec)

../../annotations/vad/radio-MUV-204427.csv
[(0, 18.950511945392492, 'speech'), (18.950511945392492, 20.076791808873722, 'nonspeech'), (20.076791808873722, 60.032, 'speech')]
../../annotations/vad/radio-FMU-135816.csv
[(0, 35.503412969283275, 'nonspeech'), (35.503412969283275, 40.75938566552901, 'speech'), (40.75938566552901, 55.75938566552901, 'nonspeech'), (55.75938566552901, 60.011, 'speech')]
../../annotations/vad/tv-ART-104726.csv
[(0, 13.131399317406144, 'speech'), (13.131399317406144, 22.05631399317406, 'nonspeech'), (22.05631399317406, 25.998293515358363, 'speech'), (25.998293515358363, 27.005119453924916, 'nonspeech'), (27.005119453924916, 29.633105802047783, 'speech'), (29.633105802047783, 30.62286689419795, 'nonspeech'), (30.62286689419795, 33.77986348122867, 'speech'), (33.77986348122867, 35.04266211604096, 'nonspeech'), (35.04266211604096, 38.50682593856655, 'speech'), (38.50682593856655, 39.1382252559727, 'nonspeech'), (39.1382252559727, 40.7764505119454, 'speech'), (40.77